# Estimateurs et intervalles de confiance

## Table des matières

* [Échantillon aléatoire et loi empirique](#Échantillon-aléatoire-et-loi-empirique)
* [Estimateurs](#Estimateurs)
* [Intervalle de confiance](#Intervalle-de-confiance)
* [Exercices](#Exercices)
* [Application des concepts avec R](Séance%202%20avec%20R.ipynb)

## Échantillon aléatoire et loi empirique

### Échantillon aléatoire

Pour commencer, intéressons-nous à ce qu'est un échantillon en probabilité.
Pour cela, considérons une suite de $n$ variables aléatoires **indépendantes** $X_i$ qui suivent une même loi de probabilité $P$. Cette suite de v.a. que l'on notera $\left(X_1, \dots, X_n\right)$ forme un **échantillon aléatoire**.  La loi $P$ est ce que l'on appellera loi parente de l'échantillon.

Pour tout événement la réalisation de l'échantillon se notera $(x_1, \dots, x_n)$ où $x_i$ est la valeur de la variable aléatoire $X_i$ correspondant à l'événement en question. La réalisation d'un échantillon aléatoire forme un échantillon. 

Un échantillon est donc un ensemble de valeurs tirées de la loi parente. Différentes réalisations de l'échantillon aléatoire peuvent donc aboutir à différents échantillons.

Passons maintenant à la pratique avec *python*. Il est tout à fait possible de générer des échantillons en utilisant la librairie numpy et plus particulièrement [numpy.random](https://numpy.org/doc/stable/reference/random/index.html#module-numpy.random). Il faut tout d'abord créer un générateur de nombres pseudos aléatoires en utilisant la fonction [default_rng](https://numpy.org/doc/stable/reference/random/generator.html#numpy.random.default_rng). On peut ensuite générer des nombres aléatoires suivant une [multitude de distributions standards](https://numpy.org/doc/stable/reference/random/generator.html#distributions).

In [ ]:
# On importe numpy
import numpy as np
# On importe matplotlib
import matplotlib.pyplot as plt
# Création de widgets
from ipywidgets import interact, interactive, FloatSlider, IntSlider

# On crée un générateur de nombres aléatoires
rg = np.random.default_rng(42)

# Création d'un échantillon de taille 10 suivant une loi normale de moyenne 2 et écart-type 1
print("Mon échantillon est : ", rg.normal(loc=2, scale=1, size=6))
print("Différentes réalisations d'un échantillon aléatoire :")
for _ in range(3):
    print(rg.standard_normal(size=5))

# Création d'un widget pour visualiser l'histogramme de l'échantillon
def gaussian_sample(mu, sigma, size):
    sample = rg.normal(loc=mu, scale=sigma, size=size)
    plt.hist(sample)
    plt.show()

mu = FloatSlider(min=-3, max=3, step=0.5, continuous_update=False)
sigma = FloatSlider(min=0.1, max=5, step=0.3, value=1, continuous_update=False)
size = IntSlider(min=30, max=500, step=1, value=300, continuous_update=False)

w = interact(gaussian_sample, mu=mu, sigma=sigma, size=size)

### Loi de proabilité empirique
À partir d'un échantillon aléatoire, on définit la loi de probabilité empirique $P_n$ qui lui est associée. Considérons un intervalle $I \subseteq \mathbb{R}$. La probabilité d'appartenir à cet intervalle sera égale au pourcentage de points de l'échantillon qui lui appartiennent. Par exemple, si on considère l'échantillon suivant, $$
(2.3, 4.5, 1.2, 3.6, 7.3, 1.3, 7.5),
$$
la probabilité empirique d'appartenir à l'intervalle $[0,3]$ vaut $3/7\simeq 43\%$ car il y a 3 valeurs parmi les 7 qui appartiennent à cet intervalle.

### Moments empiriques

Comme toute loi de probabilité, la loi de probabilité empirique admet des moments que l'on appelle, sans surprise, moments empiriques. Ces moments empiriques sont des **variables aléatoires** car l'échantillon dépend de la réalisation de l'échantillon aléatoire.

* La moyenne empirique de l'échantillon aléatoire est l'espérance d'une loi uniforme où chacune des variables aléatoires $X_i$ a un même poids $1/n$.
$$
\overline{X}_n = \dfrac{1}{n} \sum_{i=1}^{n} X_i.
$$
* La variance empirique de l'échantillon aléatoire sera, suivant la même logique,    
$$
S^{'2}_n = \dfrac{1}{n} \sum_{i=1}^{n}  \left( X_i - \overline{X}_n \right)^2.
$$

On peut, une nouvelle fois, illustrer la théorie à l'aide de python. Pour calculer la moyenne empirique d'un échantillon, on utilisera [numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) et pour sa variance empirique, [numpy.var](https://numpy.org/doc/stable/reference/generated/numpy.var.html#numpy.var). La loi de probabilité empirique revient à construire un histogramme où les nombres d'apparitions sont divisés par la taille de l'échantillon. Pour construire un histogramme, il suffit d'utiliser la fonction [numpy.histogram](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html).

In [ ]:
mu = 2
sigma = 1
# On crée un échantillon
sample = rg.normal(loc=mu, scale=sigma, size=50)

print(f"La moyenne empirique de l'échantillon est {np.mean(sample)}.")
print(f"La variance empirique de l'échantillon est {np.var(sample)}.")

hist, edges = np.histogram(sample)
p = hist/sample.size
ptot = 0
for i, pi in enumerate(p):
    print(f"La probabilité empirique de l'intervalle [{round(edges[i], 3)};{round(edges[i+1], 3)}] est {round(pi, 3)}")
    ptot += round(pi, 3)
print("La probabilité empirique totale vaut bien %.2f" %ptot)

On peut également comparer la loi de probabilité empirique à la loi de probabilité parente de l'échantillon en utilisant la fonction de répartition. Par définition, la fonction de réparition empirique est
$$
F_n(k) = \dfrac{1}{n} \sum_{i=1}^{n} \mathbb{1}_{(-\infty, k]}(x_i)
$$
où
$$
\mathbb{1}_{(-\infty, k]} = 
 \begin{cases}
    1 \quad \text{si } x_i \in (-\infty, k]\\
    0 \quad \text{sinon.}
 \end{cases}
$$

Pour la calculer avec python, il suffit donc de trier l'échantillon par ordre croissant. Ensuite, la fonction de répartition empirique évaluée en $x_k$, k-ème valeur de l'échantillon trié sera juste $k/n$. Autrement dit, **pour un échantillon trié**,
$$
F_n(x_k) = \dfrac{k}{n} \quad \text{où }  x_k \text{ appartient à l'échantillon}.
$$

In [ ]:
import scipy.stats as sp_st

# On trie l'échantillon
sample_sorted = sorted(sample)
# On calcule la fonction de répartition empirique
empirical_cdf = (np.arange(len(sample)) + 1)/len(sample)
# On trace la fonction de répartition empirique
plt.step(sample_sorted, empirical_cdf, "r", label="Loi empirique")
# On trace la fonction de répartition de la loi parente
xr = np.linspace(mu-4*sigma, mu+4*sigma, 100)
plt.plot(xr, sp_st.norm.cdf(xr, mu, sigma), "k--", label="Loi parente")
plt.legend()
plt.show()

[Retour en haut](#Table-des-matières)

## Estimateurs

### Exemple introductif

Considérons une élection présidentielle, on cherche à estimer le résultat de l'élection en effectuant un sondage auprès d'un échantillon de la population.

Mathématiquement, on modélise le problème en considérant $\Omega$ comme l'ensemble des électeurs et $A \subseteq \Omega$ comme l'ensemble des électeurs favorables au président actuel. On considère $X$ la variable indicatrice de $A$, qui vaut 1 si un électeur lui est favorable et 0 sinon. La loi de probabilité de $X$ est donc connue et il s'agit de la loi de Bernoulli de paramètre $p = P(A)$ (probabilité d'être favorable).

On cherche donc à estimer le **paramètre** $\theta = p$ qu'on ne connaît pas mais on sait toutefois qu'il appartient à un ensemble $\Theta = [0, 1]$. En effet, la probabilité $p$ ne peut pas être inférieure à zéro ou supérieure à 100%.

Après avoir fait un sondage auprès de $n$ personnes, on obtient une réalisation d'un échantillon aléatoire de taille $n$ et de loi de Bernoulli. Si 35\% des personnes sont favorables, on peut estimer que $p = 0.35$. La moyenne empirique est donc, dans ce cas, l'**estimateur** du paramètre $\theta$.

### Estimateur sans biais

Avec l'exemple précédant, on remarque qu'un estimateur est donc une **variable aléatoire** car sa valeur dépend de la réalisation de l'échantillon aléatoire. Il est ainsi possible de lui associer une espérance et une variance.

Dans le cas particulier où la valeur moyenne de l'estimateur est égale à la vraie valeur du paramètre que l'on cherche à estimer, on dira que l'estimateur est **sans biais**. Mathématiquement, pour un paramètre $\theta$, et un estimateur $\hat{\theta}$
$$
E[\hat{\theta}] = \theta
$$
c'est à dire, qu'en moyenne, l'estimateur ne se trompe pas.


Considérons maintenant une variable aléatoire $X$ de loi de probabilité $P$. L'espérance de $X$, qui est sa moyenne théorique, sera notée $m$ et sa variance sera notée $\sigma^2$.

Si on cherche à estimer $m$ et $\sigma^2$ à l'aide d'un échantillon aléatoire $(X_1, \dots, X_n)$, on pense naturellement à utiliser la moyenne et la variance empirique. Pour la moyenne empirique,
$$
\overline{X}_n = \dfrac{1}{n} \sum_{i=1}^{n} X_i.
$$
Or, on sait que chaque variable aléatoire $X_i$ suit la même loi $P$ de moyenne $m$. On a donc
\begin{align*}
E\left(\overline{X}_n\right) &= E\left( \dfrac{1}{n} \sum_{i=1}^{n} X_i \right)\\
    &= \dfrac{1}{n} \sum_{i=1}^{n} E(X_i)\\
    &= \dfrac{1}{n} \sum_{i=1}^{n} m.
\end{align*}
Ainsi on voit que **la moyenne empirique est un estimateur sans biais de la moyenne théorique** car,
$$
E\left(\overline{X}_n\right) = m
$$

Par contre, on peut montrer que la variance empirique n'est pas un estimateur sans biais de la variance théorique. En effet, on a
$$
E\left(S^{'2}_n\right) = \dfrac{n-1}{n} \sigma^2.
$$

Pour construire un estimateur sans biais de la variance théorique, on utilise la **variance empirique corrigée**,
$$
S^{2}_n = \dfrac{n}{n-1} S^{'2}_n = \dfrac{1}{n-1} \sum_{i=1}^{n} (X_i - \overline{X}_n)^2.
$$
Pour comprendre ce résultat, on remarque que les variables aléatoires de l'échantillon sont liées par la relation suivante,
$$
\overline{X}_n = \dfrac{1}{n} \sum_{i=1}^{n} X_i \Rightarrow \sum_{i=1}^{n} (X_i - \overline{X}_n) = 0
$$
et il y a donc seulement $n-1$ termes indépendants dans cette somme.

N.B. : Même si $S^{2}_n$ est un estimateur sans biais de la variance, $S_n = \sqrt{S^{2}_n}$ n'est pas un estimateur sans biais de l'écart-type. Il n'existe pas de forme analytique pour calculer l'estimateur sans biais de l'écart-type.


Considérons à présent l'utilisation de ces concepts avec python. Comme nous l'avons vu précedemment, la moyenne et la variance empirique peuvent être calculées via [numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html) et [numpy.var](https://numpy.org/doc/stable/reference/generated/numpy.var.html#numpy.var). Pour passer à la variance empirique corrigée, il suffit de jouer avec l'argument `ddof` qui représente le nombre de degrés de liberté à retirer, soit 1 dans notre cas.

In [ ]:
sample = rg.normal(loc=mu, scale=sigma, size=100)

print(f"La moyenne empirique vaut : {np.mean(sample)}")
print(f"La variance empirique corrigée vaut : {np.var(sample, ddof=1)}")

Voyons maintenant comment évolue la l'espérance de la moyenne empirique avec le nombre de réalisation de l'échantillon aléatoire.

In [ ]:
nreal = 5000
empir_means = np.zeros(nreal)
expected_mean = np.zeros(nreal)
for i in range(nreal):
    sample = rg.normal(mu, sigma, size=10)
    empir_means[i] = np.mean(sample)
    expected_mean[i] = np.mean(empir_means[:i+1])
plt.plot(np.arange(1, nreal+1, 1), expected_mean, "r")
plt.axhline(mu, color="k")
plt.xlabel("Nombre de réalisations")
plt.ylabel("Espérance de la moyenne empirique")
print("L'espérance de la moyenne empirique vaut %.2f après %.d réalisations." % (expected_mean[-1], nreal))
plt.show()

On remarque ainsi que la moyenne empirique est bien un estimateur sans biais de la moyenne. On peut faire de même avec la variance empirique et la variance empirique corrigée.

In [ ]:
nreal = 5000
empir_vars = np.zeros(nreal)
expected_var = np.zeros(nreal)
corrected_empir_vars = np.zeros(nreal)
expected_corrected_var = np.zeros(nreal)
for i in range(nreal):
    sample = rg.normal(mu, sigma, size=10)
    empir_vars[i] = np.var(sample)
    expected_var[i] = np.mean(empir_vars[:i+1])
    corrected_empir_vars[i] = np.var(sample, ddof=1)
    expected_corrected_var[i] = np.mean(corrected_empir_vars[:i+1])
plt.plot(np.arange(1, nreal+1, 1), expected_var, "b", label=r"$E(S'^2)$")
plt.plot(np.arange(1, nreal+1, 1), expected_corrected_var, "r", label=r"$E(S^2)$")
plt.axhline(sigma**2, color="k")
plt.xlabel("Nombre de réalisations")
plt.ylabel(r"Espérance de $S'^2$ et $S^2$")
plt.legend()
print("L'espérance de la variance empirique vaut %.2f après %.d réalisations." % (expected_var[-1], nreal))
print("L'espérance de la variance empirique corrigée vaut %.2f après %.d réalisations." % (expected_corrected_var[-1], nreal))
plt.show()

Il est ainsi clair que la variance empirique n'est pas un estimateur sans biais de la variance théorique. Toutefois, plus l'échantillon aléatoire sera grand, plus la différence entre variance empirique et variance empirique corrigée sera faible. En effet, les deux grandeurs ne diffèrent que d'un rapport $n/(n-1)$ qui tend vers 1 lorsque $n$ tend vers l'infini.

[Retour en haut](#Table-des-matières)

## Intervalle de confiance

### Exemple introductif

Considérons le cas d'un industriel qui commande un lot de tiges métalliques. Il ne peut utiliser ces tiges que si elles ont une longueur comprise entre 23.6 et 23.7 cm. Ces tiges étant fabriquées par une machine, leur taille peut être paramétrée. Une fois réglée avec le paramètre $m$, la taille des tiges peut être modélisée par une variable aléatoire $X$ qui suit une loi normale de moyenne $m$ et d'écart-type $\sigma$. L'écart-type est donné par le fabricant de la machine et vaut $\sigma$ = 0.02 cm.

À la réception des tiges, l'industriel veut savoir si la machine a été correctement réglée (sur une valeur m=23.65).

Il sélectionne donc $n$ tiges dans son lot pour estimer la valeur moyenne et mesure $\overline{x}_n$ = 23.63 cm. Comme il a des notions de statistique, il sait que cette valeur obtenue n'est pas la vraie moyenne $m$ de la machine mais doit s'en approcher. 

Sans prendre de risque, on peut dire que, pour $a$ et $b$ à déterminer, il y a 95\% de chances que $m$ soit comprise entre $\overline{x}_n - a$ et $\overline{x}_n + b$.

Le problème est donc de fixer les valeurs de $a$ et $b$ qui correspondent à ces «95\% de chances». L'intervalle que l'on obtient sera **l'intervalle de confiance** et la probabilité qu'à $m$ de lui appartenir sera le **niveau de confiance**.


Un intervalle de confiance de niveau de confiance $1 - \alpha \in~]0,1[$ pour un paramètre $\theta$, est un intervalle qui a la probabilité $1 - \alpha$ de contenir la vraie valeur de $\theta$.


Dans le cas de l'exemple précédent, on a
$$
P \lbrace \overline{X}_n - a < m < \overline{X}_n + b \rbrace = 1 - \alpha
$$
qui est équivalent à
$$
P \lbrace -b < \overline{X}_n - m < + a \rbrace = 1 - \alpha.
$$
Comme la loi normale est symétrique par rapport à sa moyenne, on choisi $b=a$. De plus, en divisant par $\sigma/\sqrt{n}$, l'écart-type de la moyenne empirique, on se ramène à une variable centrée réduite,
$$
 P \bigg \lbrace -\dfrac{a}{\sigma/\sqrt{n}} < \dfrac{\overline{X}_n - m}{\sigma/\sqrt{n}} < \dfrac{a}{\sigma/\sqrt{n}} \bigg \rbrace = 1 - \alpha.
$$

En réécrivant autrement,
$$
P \bigg \lbrace \Big \lvert \dfrac{  \overline{X}_n - m }{\sigma/\sqrt{n}} \Big \rvert <  \dfrac{a}{\sigma/\sqrt{n}}  \bigg \rbrace = 1 - \alpha
$$
et on voit que si $\Phi$ est la fonction de répartition de la loi normale centrée réduite, on a
$$
1 - \alpha = 2 \Phi \left( \dfrac{a}{\sigma/\sqrt{n}} \right) - 1.
$$
En utilisant ce que l'on a vu à la séance passée,
$$
 \Phi \left( \dfrac{a}{\sigma/\sqrt{n}} \right) = 1 - \dfrac{\alpha}{2}.  
$$
On est donc amené à chercher le fractile d'ordre $1 - \dfrac{\alpha}{2}$ de la loi normale centrée réduite qui vaut $1.96$. 

De retour à notre exemple, si on considère un échantillon de 100 tiges et un niveau de confiance de 95%, on trouve
$$
23.626 < m < 23.634   
$$
et on peut dire avec 95\% de certitude que la machine n'est pas réglée de manière optimale.


### Intervalle de confiance pour la moyenne
On a donc vu que l'intervalle de confiance $I_{\alpha}$, de niveau de confiance $1 - \alpha$, pour la moyenne d'une loi normale d'écart-type connu, est donné par
$$
I_{\alpha} = \left[ \overline{X}_n - u_{1-\alpha/2} \dfrac{\sigma}{\sqrt{n}} ; \overline{X}_n + u_{1-\alpha/2} \dfrac{\sigma}{\sqrt{n}} \right]
$$
où $u_{1-\alpha/2}$ est le fractile d'ordre $1-\alpha/2$ de la loi normale centrée réduite.

**Dans le cas où** l'on doit estimer la moyenne alors que **la variance de la loi normale n'est pas connue**, on peut montrer que l'intervalle $I_{\alpha}$ est donné par,
$$
I_{\alpha} = \left[ \overline{X}_n - t_{n-1, 1-\alpha/2} \dfrac{S_n}{\sqrt{n}} ; \overline{X}_n + t_{n-1, 1-\alpha/2} \dfrac{S_n}{\sqrt{n}} \right]
$$
où $t_{n-1, 1-\alpha/2}$ est le fractile d'ordre $1-\alpha/2$ de la loi de Student à $n-1$ degrés de libertés et $S_n$ est l'écart-type associé à la variance corrigée (estimateur non biaisé de la variance théorique). Le fractile de la loi de Student peut-être obtenu en utilisant la [table qui se trouve sur moodle](https://moodle.umons.ac.be/pluginfile.php/335048/mod_folder/content/0/Table%20de%20la%20Loi%20de%20Student.pdf?forcedownload=1). Lorsque le nombre de degrés de liberté devient grand (voir table), la loi de Student converge vers la loi normale et on peut donc se ramener à l'utilisation de [la table de la loi normale](https://moodle.umons.ac.be/pluginfile.php/335048/mod_folder/content/0/Table%20de%20Loi%20Normale.pdf?forcedownload=1) vue à la séance précédente.

N.B. : Il est important de noter que 
$$
u_{1-\alpha/2} = \dfrac{a}{\sigma/\sqrt{n}} \Rightarrow a = \dfrac{u_{1-\alpha/2} ~ \sigma}{\sqrt{n}}.
$$
La borne $a$ sera donc de plus en plus grande si on diminue $\alpha$. Autrement dit, au plus on voudra être sûr que la vraie valeur soit comprise dans l'intervalle, au plus l'intervalle sera grand. Par contre, au plus on utilisera un grand échantillon, au plus notre intervalle pourra être petit. En passant d'un échantillon de taille 10 à un échantillon de taille 1000, la valeur de $a$ va être divisée par 10. Finalement, plus l'écart-type de la distribution est grand, plus notre intervalle sera grand car les fluctuations autour de la vraie moyenne deviennent plus grande.

Il est également possible de calculer des intervalles de confiance avec python. Par exemple, pour obtenir celui de la moyenne dans le cas d'une distribution normale dont on connait l'écart-type, on utilisera la méthode `interval` de [scipy.stats.norm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy-stats-norm). On peut réaliser notre exemple introductif.

In [ ]:
confidence_interval = sp_st.norm.interval(0.95, loc=23.63, scale=0.02/np.sqrt(100))
print("L'intervalle de confiance de niveau 95%% pour la moyenne est : [%.3f; %.3f]" % confidence_interval)

Si on ne connait pas l'écart-type, il faudra utiliser la méthode `interval` de [scipy.stats.t](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html#scipy-stats-t) car le fractile utilisé sera celui de la loi de student.

In [ ]:
sample = np.random.normal(mu, sigma, size=100)

confidence_interval = sp_st.t.interval(0.99, df=sample.size-1, loc=np.mean(sample), scale=sp_st.sem(sample))
print("L'intervalle de confiance de niveau 99%% pour la moyenne est : [%.3f; %.3f]" % confidence_interval)

Dans ce dernier exemple, la fonction [`sem`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html#scipy-stats-sem) calcule l'écart-type de la moyenne empirique ($\sigma/\sqrt{n}$).

[Retour en haut](#Table-des-matières)

## Exercices

### Exercice 1

On considère une variable aléatoire $X$ qui représente le diamètre de sphères produites par une machine. Un échantillon de 5 sphères donne les mesures suivantes pour le diamètre (en cm): $6.33, 6.37, 6.36, 6.32, 6.37$. Déterminez un estimateur non biaisé de :

* la moyenne
* la variance
de la variable aléatoire $X$.

On pose la même question avec les mesures suivantes : $8.3, 10.6, 9.7, 8.8, 10.2, 9.4$. Comparez la variance empirique et l'estimateur non biaisé de la variance.

### Exercice 2

100 étudiantes (d'une université Belge comptant 1546 étudiantes) prises au hasard ont leur taille répartie de la manière suivante :

Taille (m) | Nombre d'étudiante
-----------|--------------------
1.57 | 5
1.63 | 18
1.69 | 42
1.75 | 27
1.81 | 8

Soit $X$ une variable aléatoire modélisant la taille d'une étudiante. Déterminez un estimateur non biaisé de la moyenne et de la variance de $X$.


### Exercice 3

On considère un appareil électrique sur batterie. On sélectionne un échantillon de 10 appareils et on obtient que la moyenne empirique de la durée de vie de la batterie est de 1200 heures et son écart-type empirique de 100 heures. Déterminez un estimateur non biaisé de la moyenne et de la variance.
      
On suppose maintenant que ce résultat a été obtenu sur un échantillon de 30, 50 ou 100 portables. Recalculez les éstimateurs correspondants. Que pouvez-vous conclure ?

### Exercice 4

Supposons que la distribution des tailles des étudiants est normale de moyenne $\mu$ = 175 cm et d'écart-type $\sigma$ = 8cm.
1. Si l'on tire plusieurs échantillons de taille $n = 4$ et que l'on calcule à chaque fois la moyenne de l'échantillon, comment fluctuent ces moyennes ?
2. Si on quadruple la taille de l'échantillon, comment varie l'erreur d'estimation, c'est-à-dire l'écart-type de la moyenne empirique ?

### Exercices 5 et 6

* La moyenne empirique des mesures du temps de réaction d'une  population de 200 conducteurs pris hasard durant une semaine en Belgique est de 0.824 s et son écart-type est de 0.042 s. Trouvez l'intervalle de confiance de la moyenne empirique pour un niveau de confiance de 95% puis de 99%.
* Si on considère maintenant que l'écart-type du temps de réaction est de 0.05 s. De combien doit être la taille de l'échantillon pour que l'erreur sur la moyenne du temps de réaction soit inférieur à 0.01 s avec un niveau de confiance de 95% ? Même question pour un niveau de confiance de 99%.

### Exercice 7

On suppose que la durée des achats dans un grand magasin suit une loi de distribution normale $N(m , \sigma^2=256)$ (min). On tire un échantillon aléatoire de 64 personnes parmi les clients effectuant leurs achats un jour déterminé. La moyenne des durées des achats mesurée sur cet échantillon est de 33 minutes. Trouvez un intervalle de confiance pour la moyenne des durées des achats de l'ensemble des clients fréquentant ce grand magasin aux niveaux d'incertitude $\alpha$ = 0.05 et $\alpha$ = 0.01.


### Exercices 8 et 9

* On suppose que, dans une classe très nombreuse, les notes du Prof. De Coninck en statistique sont distribuées suivant une distribution $N(m,\sigma^2)$. On extrait un échantillon de 4 notes : 64, 66, 89, 77. Calculez un intervalle de confiance pour un niveau de confiance de 95% pour la moyenne des notes de toute la classe.
* Dans une province, un candidat $D$ a les faveurs de la population à 55%. Trouvez le nombre de votants nécessaires pour être « sûr » à  95% puis 99% que le candidat D soit élu.

[Retour en haut](#Table-des-matières)